In [30]:
from pyspark.sql import SparkSession
import sqlite3
import pandas as pd
import numpy as np
from pyspark.sql.functions import *

In [2]:
"""
Connecting to SQLite database
"""

conn = sqlite3.connect("database.sqlite")
cur = conn.cursor()

In [3]:
"""
Reading tables and storing them into pandas dataframe
"""

ply_attr=pd.read_sql_query("Select * from Player_Attributes",conn)
ply=pd.read_sql_query("Select * from Player",conn)
country=pd.read_sql_query("Select * from Country",conn)
league=pd.read_sql_query("Select * from League",conn)
match=pd.read_sql_query("Select * from Match",conn)
team=pd.read_sql_query("Select * from Team",conn)
team_attr=pd.read_sql_query("Select * from Team_Attributes",conn)

In [4]:
spark = SparkSession \
    .builder \
    .appName("Soccer") \
    .getOrCreate()

In [5]:
plyr_attr=spark.createDataFrame(ply_attr)
plyr=spark.createDataFrame(ply)
coun=spark.createDataFrame(country)
leag=spark.createDataFrame(league)
mat=spark.createDataFrame(match)
teams=spark.createDataFrame(team)
team_attrs=spark.createDataFrame(team_attr)

In [25]:
xply=plyr_attr.select(plyr_attr.columns[:6])
yply=plyr_attr.select(['id','stamina','strength'])

xply=xply.alias('xply')
yply=yply.alias('yply')

df_plyr=xply.join(yply,xply.id==yply.id).select("xply.id","player_fifa_api_id","player_api_id","date","overall_rating", \
                                               "potential","stamina","strength")

In [46]:
#df_plyr.select([(when(isnull(c), c)).alias(c) for c in df_plyr.columns]).show()
df_plyr.where(col('stamina').isNull()).show()

+---+------------------+-------------+----+--------------+---------+-------+--------+
| id|player_fifa_api_id|player_api_id|date|overall_rating|potential|stamina|strength|
+---+------------------+-------------+----+--------------+---------+-------+--------+
+---+------------------+-------------+----+--------------+---------+-------+--------+

